In [58]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
import pandas as pd

# 加载数据
file_path = '/kaggle/input/text-classificatiopn/news.csv'
data = pd.read_csv(file_path)

# 显示数据的前几行
data.head()


,label,text
0,教育,澳移民子女成长记：带着中国心融入主流社会新华网悉尼5月31日电 无论哪个国家的父辈与子女间都...
1,体育,快船vs火箭首发：休城旨在练兵 小德帕特森进先发新浪体育讯北京时间4月10日消息，在常规赛还...
2,科技,3英寸屏高清闪存DV 三洋TH1特价1499 作者：中关村在线 飘雪 ...
3,时尚,贝嫂乏味归乏味 还有人买账Victoria Beckham 乏味归乏味 还有人买账大姐大的阵...
4,房产,三亚岭南赶房集 金九银十再兴购房游纯粹的旅行闲适有余却“收获”不足，设计一条可以兼容曼妙风景...


In [62]:
# 使用LabelEncoder将标签转换为数值形式
label_encoder = LabelEncoder()
data["label_num"] = label_encoder.fit_transform(data["label"])

In [63]:
# 获取标签和数值之间的对应关系
label_mapping = dict(zip(data["label"], data["label_num"]))

print("标签到数值的映射:", label_mapping)

标签到数值的映射: {'教育': 4, '体育': 0, '科技': 8, '时尚': 5, '房产': 3, '家居': 2, '财经': 9, '时政': 6, '娱乐': 1, '游戏': 7}


In [3]:
from sklearn.preprocessing import LabelEncoder

# 将标签转换为数值形式
label_encoder = LabelEncoder()
data["label"] = label_encoder.fit_transform(data["label"])

In [6]:
!pip install datasets



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.6 MB/s eta 0:00:00







     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00






     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.8 MB/s eta 0:00:00



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
























In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

2024-04-17 07:06:48.796490: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-17 07:06:48.796627: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-17 07:06:48.931877: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
dataset = load_dataset("csv", data_files="/content/drive/MyDrive/data/news.csv", split="train")
dataset

Dataset({
    features: ['label', 'text'],
    num_rows: 1000
})

In [5]:
from datasets import Dataset

dataset = Dataset.from_pandas(data)
dataset

Dataset({
    features: ['label', 'text'],
    num_rows: 1000
})

In [6]:
dataset = dataset.filter(lambda x: x["text"] is not None)
dataset

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'text'],
    num_rows: 1000
})

In [7]:
datasets = dataset.train_test_split(test_size=0.2)
datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 800
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 200
    })
})

In [8]:
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [9]:

def process_function(examples):
    tokenized_examples = tokenizer(examples["text"], max_length=128, truncation=True)
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples

tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 800
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 200
    })
})

In [10]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-chinese", num_labels=10)

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.config

BertConfig {
  "_name_or_path": "bert-base-chinese",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8,
    "LABEL_9": 9
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers":

In [11]:
model.config.num_labels

10

In [13]:
!pip install evaluate


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.4 MB/s eta 0:00:00


In [14]:
import evaluate

acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

In [39]:
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    return acc

In [40]:
train_args = TrainingArguments(output_dir="./checkpoints",      # 输出文件夹
                               per_device_train_batch_size=64,  # 训练时的batch_size
                               per_device_eval_batch_size=128,  # 验证时的batch_size
                               logging_steps=10,                # log 打印的频率
                               evaluation_strategy="epoch",     # 评估策略
                               save_strategy="epoch",           # 保存策略
                               save_total_limit=3,              # 最大保存数
                               learning_rate=2e-5,              # 学习率
                               weight_decay=0.01,               # weight_decay
                               metric_for_best_model="accuracy",      # 设定评估指标
                               load_best_model_at_end=True,      # 训练完成后加载最优模型
                               report_to=['tensorboard']
                              )    
train_args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_

In [41]:
from transformers import DataCollatorWithPadding
trainer = Trainer(model=model,
                  args=train_args,
                  train_dataset=tokenized_datasets["train"],
                  eval_dataset=tokenized_datasets["test"],
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                  compute_metrics=eval_metric)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [42]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.067200,0.087545,0.980000
2,0.059600,0.072355,0.980000
3,0.036700,0.065299,0.980000


TrainOutput(global_step=39, training_loss=0.04902342343941713, metrics={'train_runtime': 39.8211, 'train_samples_per_second': 60.269, 'train_steps_per_second': 0.979, 'total_flos': 157877972582400.0, 'train_loss': 0.04902342343941713, 'epoch': 3.0})

In [43]:
trainer.evaluate(tokenized_datasets["test"])

{'eval_loss': 0.08754459023475647,
 'eval_accuracy': 0.98,
 'eval_runtime': 0.8151,
 'eval_samples_per_second': 245.368,
 'eval_steps_per_second': 2.454,
 'epoch': 3.0}

In [44]:
trainer.predict(tokenized_datasets["test"])

PredictionOutput(predictions=array([[-1.1238894 , -1.1777285 , -0.5195459 , ...,  4.829511  ,
        -0.35051307, -0.80929947],
       [-0.01959847,  5.07517   , -1.028982  , ..., -0.9674754 ,
        -0.14434259, -0.29346186],
       [-0.78315926, -1.1029011 , -0.8485481 , ..., -0.5813239 ,
        -1.188333  , -0.8094628 ],
       ...,
       [-0.14287795, -0.9206116 ,  0.17444097, ..., -0.49079615,
        -0.5668939 ,  5.0414214 ],
       [-0.6064959 , -0.51849574, -0.44860598, ..., -0.15690486,
        -0.8818646 , -1.3322617 ],
       [-0.55834967, -1.1203219 ,  4.8613696 , ..., -0.13031597,
        -0.17338564, -0.3073368 ]], dtype=float32), label_ids=array([7, 1, 6, 1, 8, 3, 0, 4, 2, 1, 1, 6, 7, 2, 6, 8, 5, 3, 5, 0, 2, 6,
       1, 8, 6, 4, 9, 8, 7, 1, 5, 3, 6, 2, 1, 9, 9, 2, 0, 1, 3, 3, 0, 6,
       4, 6, 8, 5, 6, 6, 6, 7, 4, 0, 5, 0, 2, 9, 6, 8, 4, 0, 6, 9, 5, 0,
       6, 8, 6, 9, 2, 2, 9, 6, 2, 5, 4, 8, 3, 4, 2, 2, 0, 0, 4, 5, 8, 1,
       1, 8, 3, 6, 8, 4, 0, 5, 9, 8, 1, 

In [46]:
from transformers import pipeline

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [68]:
id2label_dic={}
for k,v in label_mapping.items():
    id2label_dic[v] = k

In [69]:
id2label_dic

{4: '教育',
 0: '体育',
 8: '科技',
 5: '时尚',
 3: '房产',
 2: '家居',
 9: '财经',
 6: '时政',
 1: '娱乐',
 7: '游戏'}

In [71]:
model.config.id2label = id2label_dic
sen = "快船vs火箭首发：休城旨在练兵 小德帕特森进先发新浪体育讯北京时间4月10日消息，在常规赛还剩下三场的时候，火箭已经彻底的失去了进军季后赛的希望。所以今天战快船旨在练兵，再加上洛瑞一直有伤，帕特森和德拉季奇则接替小钢炮和斯科拉首发出场。以下为双方本场比赛的首发阵容：快船：威廉姆斯、戈登、穆恩、格里芬、乔丹火箭：德拉季奇、马丁、巴丁格、帕特森、海耶斯(新体)！"
pipe(sen)

[{'label': '体育', 'score': 0.9791619777679443}]